In [1]:
from mesmerize_core import *
import numpy as np
from copy import deepcopy
import pandas as pd

2023-01-08 01:50:46.011990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 01:50:46.095245: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 01:50:46.460932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib64
2023-01-08 01:50:46.460985: W tensorflow/stream_executor/platform/default/dso_

**You will need `fastplotlib` installed for the visualizations**

In [2]:
from fastplotlib import ImageWidget
from ipywidgets import VBox, IntSlider, Layout

In [3]:
pd.options.display.max_colwidth = 120

# Paths
These are the only variables you will need to modify in this demo notebook. You will need to set the paths according to your own `caiman_data` dir path

Explanation:

`set_parent_raw_data_path()` - This function from `mesmerize_core` sets the **top level raw data directory**. It is generally the top level directory for your raw experimental data. This allows you to move your experiment directory structure between computers, as long as you keep everything under the parent path the same.

For example,

On Linux based systems if you have your experimental data in the following dir:

`/data/my_name/exp_top_level/....`

You could set `/data/my_name` as the "parent raw data path", and you can then move `exp_top_level/...` between computers.

On windows:

`D:/my_name/exp_top_level/...`

You could set `D:/my_name` as the parent raw data path, and you can then move `exp_top_level/...` between computers.

In [4]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/home/kushalk/caiman_data/")

PosixPath('/home/kushalk/caiman_data')

### Batch path, this is where caiman outputs will be organized

This can be anywhere, it does not need to be under the parent raw data path.

**We recommend using [pathlib](https://docs.python.org/3/library/pathlib.html) instead of manually managing paths as strings. `pathlib` is just a part of the Python standard library, it makes it much easier to deal with paths and saves a lot of time in the long-run! It also makes your paths compatible across operating systems.**

In [5]:
batch_path = get_parent_raw_data_path().joinpath("mesmerize-batch/batch.pickle")

# Create a new batch

This creates a new pandas `DataFrame` with the columns that are necessary for mesmerize. In mesmerize we call this the **batch DataFrame**. You can add additional columns relevant to your experiment, but do not modify columns used by mesmerize.

Note that when you create a DataFrame you will need to use `load_batch()` to load it later. You cannot use `create_batch()` to overwrite an existing batch DataFrame

In [6]:
# create a new batch
df = create_batch(batch_path)
# to load existing batches use `load_batch()`
# df = load_batch(batch_path)

# View the dataframe

It is empty with the appropriate columns for mesmerize

In [7]:
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid


# Path to the input movie

An input movie must be anywhere within `raw data path` or `batch path`. We will use the Sue 2p example.

In [8]:
movie_path = get_parent_raw_data_path().joinpath("example_movies/Sue_2x_3000_40_-46.tif")

# Motion correction parameters

Parameters for all algos have the following structure:

```python
{"main": {... params directly passed to caiman}}
```

In [9]:
# We will start with one version of parameters
mcorr_params1 =\
{
  'main': # this key is necessary for specifying that these are the "main" params for the algorithm
    {
        'max_shifts': [24, 24],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# Add a "batch item", this is the combination of:
* algorithm to run, `algo`
* input movie to run the algorithm on, `input_movie_path`
* parameters for the specified algorithm, `params`
* a name for you to keep track of things, usually the same as the movie filename, `item_name`

In [10]:
# add an item to the batch
df.caiman.add_item(
    algo='mcorr',
    input_movie_path=movie_path,
    params=mcorr_params1,
    item_name=movie_path.stem,  # filename of the movie, but can be anything
)

df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",None,None,63323098-88ff-4298-9bbe-e2e3f3e523f6


## We can now see that there is one item, a.k.a. row or pandas `Series`, in the batch dataframe, we can add another item with the same input movie but with different parameters.

### **When adding batch items with the same `input_movie_path` (i.e. same input movie but different parameters) it is useful to give them the same `item_name`.**

In [11]:
# We create another set of params, useful for gridsearches for example
mcorr_params2 =\
{
  'main':
    {
        'max_shifts': [24, 24],
        'strides': [24, 24],
        'overlaps': [12, 12],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# add other param variant to the batch
df.caiman.add_item(
  algo='mcorr',
  item_name=movie_path.stem,
  input_movie_path=movie_path,
  params=mcorr_params2
)

df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",None,None,63323098-88ff-4298-9bbe-e2e3f3e523f6
1,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",None,None,1be262f5-1186-47a5-8827-b4b488bae3ff


## We can see that there are two batch items for the same input movie.

### We can also use a `for` loop to add multiple different parameter variants more efficiently.

In [12]:
# copy the mcorr_params2 dict to make some changes
new_params = deepcopy(mcorr_params2)

# some variants of max_shifts
for shifts in [1, 6, 12]: 
    # deep copy is the safest way to copy dicts
    new_params = deepcopy(new_params)
    
    # assign the "max_shifts"
    new_params["main"]["max_shifts"] = (shifts, shifts)
    
    df.caiman.add_item(
      algo='mcorr',
      item_name=movie_path.stem,
      input_movie_path=movie_path,
      params=new_params
    )

In [13]:
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",None,None,63323098-88ff-4298-9bbe-e2e3f3e523f6
1,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",None,None,1be262f5-1186-47a5-8827-b4b488bae3ff
2,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (1, 1), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",None,None,d33e6387-1343-4d38-af0b-6ef35bf027c0
3,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",None,None,2207c2b5-a722-4fa7-9380-6d930b82917e
4,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (12, 12), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",None,None,78d016f0-4d6f-4896-a143-8f201cd9a153


## Now we can see that there are many parameter variants, but it is not easy to see the differences in parameters between the rows that have the same `item_name`.

### We can use the `caiman.get_params_diffs()` extension to see the unique parameters between rows with the same `item_name`

In [14]:
diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])
diffs

/tmp/ipykernel_304295/1437176290.py:1: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])


0    {'overlaps': (24, 24), 'max_shifts': (24, 24), 'strides': (48, 48)}
1    {'overlaps': (12, 12), 'max_shifts': (24, 24), 'strides': (24, 24)}
2      {'overlaps': (12, 12), 'max_shifts': (1, 1), 'strides': (24, 24)}
3      {'overlaps': (12, 12), 'max_shifts': (6, 6), 'strides': (24, 24)}
4    {'overlaps': (12, 12), 'max_shifts': (12, 12), 'strides': (24, 24)}
Name: params, dtype: object

# Indexing rows and running batch item(s)
#### You can run a single batch item by calling `caiman.run()` on a `Series` (row) of the DataFrame. One way to get the row is integer indexing using `df.iloc[index]`

In [15]:
# get the first batch item
row = df.iloc[0]

### You can see how the various `pandas.Series` extensions are accessible at the level of dataframe rows.

Move the cursor to the end of the following line and press `Tab` on your keyboard. You can select the `caiman.run()` function and press `Shift + Tab` to see the docstring. You can also instead refer to the API docs. https://mesmerize-core.readthedocs.io/en/latest/api/common.html#mesmerize_core.CaimanSeriesExtensions

Note tab completion doesn't work if you use `df.iloc[i].caiman.<method_name>`, you need to apply the indexer to see the docstring.

In [ ]:
row.caiman.

# Run a single batch item

Run the row that we have selected above, on Linux & Mac it will run in subprocess but on Windows it will run in the local kernel. If using the subprocess backend you can use `run(wait=False)` to not block the current kernel when running.

In [16]:
# run the first "batch item"
# this will run in a subprocess by default on Linux & Mac
# on windows it will run locally
process = row.caiman.run()

# reload dataframe from disk when done
df = df.caiman.reload_from_disk()

2023-01-08 01:51:12.899528: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 01:51:12.982279: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 01:51:13.344653: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

starting mc


100%|██████████| 1/1 [00:00<00:00,  8.86it/s]


Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/63323098-88ff-4298-9bbe-e2e3f3e523f6/63323098-88ff-4298-9bbe-e2e3f3e523f6-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/63323098-88ff-4298-9bbe-e2e3f3e523f6/63323098-88ff-4298-9bbe-e2e3f3e523f6-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/63323098-88ff-4298-9bbe-e2e3f3e523f6/63323098-88ff-4298-9bbe-e2e3f3e523f6-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
mc finished successfully!
computing projections
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/63323098-88ff-4298-9bbe-e2e3f3e523f6/63323098-88ff-4298-9bbe-e2e3f3e523f6-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/63323098-88ff-4298-9bbe-e2e3f3e523f6/6332

# Use a loop to run multiple batch items.

`df.iterrows()` iterates through rows and returns the numerical index and row for each iteration

In [17]:
for i, row in df.iterrows():
    if not i > 0: # skip the first item since we've run it already
        continue
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

2023-01-08 01:51:22.114614: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 01:51:22.193581: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 01:51:22.551171: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

starting mc


100%|██████████| 1/1 [00:00<00:00, 12.85it/s]

100%|██████████| 1/1 [00:00<00:00,  8.67it/s]


Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/1be262f5-1186-47a5-8827-b4b488bae3ff/1be262f5-1186-47a5-8827-b4b488bae3ff-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/1be262f5-1186-47a5-8827-b4b488bae3ff/1be262f5-1186-47a5-8827-b4b488bae3ff-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/1be262f5-1186-47a5-8827-b4b488bae3ff/1be262f5-1186-47a5-8827-b4b488bae3ff-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
mc finished successfully!
computing projections
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/1be262f5-1186-47a5-8827-b4b488bae3ff/1be262f5-1186-47a5-8827-b4b488bae3ff-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/1be262f5-1186-47a5-8827-b4b488bae3ff/1be2

2023-01-08 01:51:33.832436: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 01:51:33.917692: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 01:51:34.288143: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

...

Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/2207c2b5-a722-4fa7-9380-6d930b82917e/2207c2b5-a722-4fa7-9380-6d930b82917e-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/2207c2b5-a722-4fa7-9380-6d930b82917e/2207c2b5-a722-4fa7-9380-6d930b82917e-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/2207c2b5-a722-4fa7-9380-6d930b82917e/2207c2b5-a722-4fa7-9380-6d930b82917e-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
mc finished successfully!
computing projections
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/2207c2b5-a722-4fa7-9380-6d930b82917e/2207c2b5-a722-4fa7-9380-6d930b82917e-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/2207c2b5-a722-4fa7-9380-6d930b82917e/2207

2023-01-08 01:51:57.052436: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 01:51:57.134944: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 01:51:57.510917: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

starting mc


100%|██████████| 1/1 [00:00<00:00,  9.04it/s]


Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/78d016f0-4d6f-4896-a143-8f201cd9a153/78d016f0-4d6f-4896-a143-8f201cd9a153-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/78d016f0-4d6f-4896-a143-8f201cd9a153/78d016f0-4d6f-4896-a143-8f201cd9a153-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/78d016f0-4d6f-4896-a143-8f201cd9a153/78d016f0-4d6f-4896-a143-8f201cd9a153-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
mc finished successfully!
computing projections
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/78d016f0-4d6f-4896-a143-8f201cd9a153/78d016f0-4d6f-4896-a143-8f201cd9a153-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/78d016f0-4d6f-4896-a143-8f201cd9a153/78d0

# Reload the DataFrame to see the outputs information for the mcorr batch item
### It is necessary to ALWAYS use `df = df.caiman.reload_from_disk()` after running a single batch item or a loop of batch items. You must not add new batch items until you reload it if you have ran items!

In [18]:
df = df.caiman.reload_from_disk()

## We can see that the `outputs` column has been filled in

In [19]:
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': 63323098-88ff-4298-9bbe-e2e3f3e523f6/63323098-88ff-4298-9bbe-e2e3f3e523f6_mean_projection.n...,None,63323098-88ff-4298-9bbe-e2e3f3e523f6
1,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': 1be262f5-1186-47a5-8827-b4b488bae3ff/1be262f5-1186-47a5-8827-b4b488bae3ff_mean_projection.n...,None,1be262f5-1186-47a5-8827-b4b488bae3ff
2,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (1, 1), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': d33e6387-1343-4d38-af0b-6ef35bf027c0/d33e6387-1343-4d38-af0b-6ef35bf027c0_mean_projection.n...,None,d33e6387-1343-4d38-af0b-6ef35bf027c0
3,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 2207c2b5-a722-4fa7-9380-6d930b82917e/2207c2b5-a722-4fa7-9380-6d930b82917e_mean_projection.n...,None,2207c2b5-a722-4fa7-9380-6d930b82917e
4,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (12, 12), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': 78d016f0-4d6f-4896-a143-8f201cd9a153/78d016f0-4d6f-4896-a143-8f201cd9a153_mean_projection.n...,None,78d016f0-4d6f-4896-a143-8f201cd9a153


# Check if the algorithm ran successfully for a item

In [20]:
# True if the algo ran succesfully
df.iloc[0]["outputs"]["success"]

True

# Visualization using `fastplotlib`
You will need `fastplotlib` installed for this, see https://github.com/kushalkolar/fastplotlib

# Get the input movie and mcorr so we can visualize them

Note that you DO NOT need to manually work with file paths. For tiff input files it returns it as a memmaped array (if possible) with lazy loading. It will try to use a mesmerize `LazyArray` if the file cannot be memmaped.

In [21]:
# you can change the index to look at the mcorr results of different batch items
index = 0

# get input movie as memmap
input_movie = df.iloc[index].caiman.get_input_movie()

# load mcorr output movie, also as a memmaped array
mcorr_movie = df.iloc[index].mcorr.get_output()

Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/63323098-88ff-4298-9bbe-e2e3f3e523f6/63323098-88ff-4298-9bbe-e2e3f3e523f6-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap


# Visualize raw & MCorr movie side-by-side

### fastplotlib `ImageWidget` to visualize raw & mcorr movie side by side

`ImageWidget` assumes `"txy"` dimension order by default for 2D movies. You can set other orders using the `dims_order` kwarg

In [22]:
mcorr_iw = ImageWidget(
    data=[input_movie, mcorr_movie], 
    vmin_vmax_sliders=True, 
    cmap="gnuplot2"
)
mcorr_iw.show()

RFBOutputContext()

# Frame averaging over a window

This makes it easier to visually inspect motion

In [23]:
# window function on the "t" (time) dimension, using mean of 17 frames
mcorr_iw.window_funcs = {"t": (np.mean, 17)}

## Close the canvas to free up GPU processing time, not necessary if you have a powerful GPU

In [24]:
mcorr_iw.plot.canvas.close()

## With `ImageWidget` you can view all 5 mcorr results simultaneously!

### This depends on your hard drive's capabilities

In [24]:
# first item is just the raw movie
movies = [df.iloc[0].caiman.get_input_movie()]

# subplot titles
subplot_names = ["raw"]

# we will use the mean images later
means = [df.iloc[0].caiman.get_projection("mean")]

# add all the mcorr outputs to the list
for i, row in df.iterrows():
    # add to the list of movies to plot
    movies.append(row.mcorr.get_output())
    
    # the unique params for this item
    params = diffs.iloc[i]
    
    # subplot title to show dataframe index
    subplot_names.append(f"ix: {i}")
    
    # mean images which we'll use later
    means.append(row.caiman.get_projection("mean"))

# create the widget
mcorr_iw_multiple = ImageWidget(
    data=movies,  # list of movies
    window_funcs={"t": (np.mean, 17)}, # window_funcs is also a kwarg
    vmin_vmax_sliders=True,
    names=subplot_names,  # subplot names used for titles
    cmap="gnuplot2"
)
mcorr_iw_multiple.show()

Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/63323098-88ff-4298-9bbe-e2e3f3e523f6/63323098-88ff-4298-9bbe-e2e3f3e523f6-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/1be262f5-1186-47a5-8827-b4b488bae3ff/1be262f5-1186-47a5-8827-b4b488bae3ff-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/d33e6387-1343-4d38-af0b-6ef35bf027c0/d33e6387-1343-4d38-af0b-6ef35bf027c0-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/2207c2b5-a722-4fa7-9380-6d930b82917e/2207c2b5-a722-4fa7-9380-6d930b82917e-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/78d016f0-4d6f-4896-a143-8f201cd9a153/78d016f0-4d6f-4896-a143-8f201cd9a153-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_

RFBOutputContext()

In [26]:
df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])

/tmp/ipykernel_286260/3787477196.py:1: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])


0    {'overlaps': (24, 24), 'max_shifts': (24, 24), 'strides': (48, 48)}
1    {'overlaps': (12, 12), 'max_shifts': (24, 24), 'strides': (24, 24)}
2      {'overlaps': (12, 12), 'max_shifts': (1, 1), 'strides': (24, 24)}
3      {'overlaps': (12, 12), 'max_shifts': (6, 6), 'strides': (24, 24)}
4    {'overlaps': (12, 12), 'max_shifts': (12, 12), 'strides': (24, 24)}
Name: params, dtype: object

### Modify the `window_funcs` at any time

In [27]:
mcorr_iw_multiple.window_funcs["t"].window_size = 7

## There is some motion on the left side of of `ix: 2` at timepoint `1452`, `2037` and a few others. This will be more obvious if we substract a mean image from each frame. You can use `frame_apply` to apply a function before displaying frames in the `ImageWidget`

This can be combined with `window_funcs` or used by itself. If used in combination with `window_funcs`, the window functions are computed first and then fed to `frame_apply`.

For this example the `frame_apply` functions subtract the mean image for each movie.

General form:

```python
{
    data_ix: function() # returns 2D frame
    ...
}
```

In [28]:
subtract_means = {
    0: lambda x: x - means[0],
    1: lambda x: x - means[1],
    2: lambda x: x - means[2],
    3: lambda x: x - means[3],
    4: lambda x: x - means[4],
    5: lambda x: x - means[5]
}

In [29]:
mcorr_iw_multiple.frame_apply = subtract_means

### Different colormaps can make the motion more obvious

In [30]:
for sp in mcorr_iw_multiple.plot:
    sp.get_graphics()[0].cmap = "jet"

In [31]:
# disable frame apply
mcorr_iw_multiple.frame_apply = dict()

# ix `3` seems to work the best so we will cleanup the DataFrame and remove all other items.

### You can remove batch items (i.e. rows) using `df.caiman.remove_item(<item_uuid>)`

**Note that this also cleans up the output data in the batch directory!**

In [32]:
# make a list of rows we want to keep using the uuids
rows_keep = [df.iloc[3].uuid]
rows_keep

['09610b86-917a-41e7-a493-2d036bf44428']

In [33]:
for i, row in df.iterrows():
    if row.uuid not in rows_keep:
        df.caiman.remove_item(row.uuid)

df

/home/kushalk/repos/mesmerize-core/mesmerize_core/caiman_extensions/common.py:206: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_children
This feature will change in the future and directly return the  a DataFrame of children (rows, ie. child batch items row) instead of a list of UUIDs

  children = self.get_children(index)
/home/kushalk/repos/mesmerize-core/mesmerize_core/caiman_extensions/common.py:206: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_children
This feature will change in the future and directly return the  a DataFrame of children (rows, ie. child batch items row) instead of a list of UUIDs

  children = self.get_children(index)
/home/kushalk/repos/mesmerize-core/mesmerize_core/caiman_extensions/common.py:206: FutureWarning: You are trying to use the following experi

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428_mean_projection.n...,None,09610b86-917a-41e7-a493-2d036bf44428


### As you can see above, the numerical index changed for what was previously item 3. Indices are always reset when you use `caiman.remove_item()`. However, UUIDs are always maintained.

# CNMF

## Continue from mcorr above and perform CNMF using the good mcorr output.

First, the params for CNMF. Put the CNMF params within the `main` key, `refit` is if you want to run CNMF for a second iteration.

In [36]:
# some params for CNMF
params_cnmf =\
{
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'fr': 30, # framerate, very important!
            'p': 1,
            'nb': 2,
            'merge_thr': 0.85,
            'rf': 15,
            'stride': 6, # "stride" for cnmf, "strides" for mcorr
            'K': 4,
            'gSig': [4, 4],
            'ssub': 1,
            'tsub': 1,
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,
            'rval_thr': 0.7,
            'use_cnn': True,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

### Add a single cnmf item to the batch

In [35]:
# add a batch item
df.caiman.add_item(
    algo='cnmf', # algo is cnmf
    input_movie_path=df.iloc[0],  # use mcorr output from a completed batch item
    params=params_cnmf,
    item_name=df.iloc[0]["item_name"], # use the same item name
)

### Just like with motion correction, we can use loops to add multiple parameter variants. This is useful to perform a parameter search to find the params that work best for your dataset. Here I will use `itertools.product` which is better than deeply nested loops.

In [37]:
from itertools import product

# variants of several parameters
gSig_variants = [6, 8]
K_variants = [4, 8]
merge_thr_variants = [0.8, 0.95]

# always use deepcopy like before
new_params_cnmf = deepcopy(params_cnmf)

# create a parameter grid
parameter_grid = product(gSig_variants, K_variants, merge_thr_variants)

# a single for loop to go through all the various parameter combinations
for gSig, K, merge_thr in parameter_grid:
    # deep copy params dict just like before
    new_params_cnmf = deepcopy(new_params_cnmf)
    
    new_params_cnmf["main"]["gSig"] = [gSig, gSig]
    new_params_cnmf["main"]["K"] = K
    new_params_cnmf["main"]["merge_thr"] = merge_thr
    
    # add param combination variant to batch
    df.caiman.add_item(
        algo="cnmf",
        item_name=df.iloc[0]["item_name"],
        input_movie_path=df.iloc[0],
        params=new_params_cnmf
    )

### See that there are a lot of new cnmf batch items

In [38]:
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428_mean_projection.n...,None,09610b86-917a-41e7-a493-2d036bf44428
1,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 't...",None,None,2b9dabd8-7d22-4327-8479-aea36bde79cf
2,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 'ts...",None,None,38f85afb-417f-453a-8da5-28753b0dc290
3,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 't...",None,None,a49689ad-f093-49ff-9a86-058835eccd34
4,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 'ts...",None,None,f67dbcb9-fd5b-4d81-acc4-f080a7f81db4
5,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 't...",None,None,ff76c624-2c29-4c2d-ba91-3eb471a8e9e3
6,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 'ts...",None,None,879d1d8b-dfe7-4b4f-aa2f-b987d83f7788
7,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 't...",None,None,539745c5-ec3e-4a63-a7c2-eb41af9a080e
8,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'ts...",None,None,da760704-f760-4f34-b26a-06eb5cc3cfec
9,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 't...",None,None,f4474d42-f6ae-4fba-a1a5-c181b4b1796e


## Since it is difficult to see the different parameter variants above, we can just view the diffs

### The index numbers on the diffs correspond to the indices in the parent DataFrame above

In [39]:
df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[1]["item_name"])

/tmp/ipykernel_286260/3330105589.py:1: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[1]["item_name"])


1    {'K': 4, 'gSig': (4, 4), 'merge_thr': 0.85}
2     {'K': 4, 'gSig': (6, 6), 'merge_thr': 0.8}
3    {'K': 4, 'gSig': (6, 6), 'merge_thr': 0.95}
4     {'K': 8, 'gSig': (6, 6), 'merge_thr': 0.8}
5    {'K': 8, 'gSig': (6, 6), 'merge_thr': 0.95}
6     {'K': 4, 'gSig': (8, 8), 'merge_thr': 0.8}
7    {'K': 4, 'gSig': (8, 8), 'merge_thr': 0.95}
8     {'K': 8, 'gSig': (8, 8), 'merge_thr': 0.8}
9    {'K': 8, 'gSig': (8, 8), 'merge_thr': 0.95}
Name: params, dtype: object

### Run the added `cnmf` batch items, they start at index 1 as we can see from the DataFrame.

In [40]:
for i, row in df.iterrows():
    if i < 1:
        continue
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = load_batch(df.paths.get_batch_path())

2023-01-08 01:31:09.620801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 01:31:09.704789: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 01:31:10.078529: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                          Sue_2x_3000_40_-46
input_movie_path    09610b86-917a-41e7-a493-2d036bf44428/09610b86-...
params              {'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...
outputs                                                          None
comments                                                         None
uuid                             2b9dabd8-7d22-4327-8479-aea36bde79cf
Name: 1, dtype: object
With params:{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 'tsub': 1, 'method_init': 'greedy_roi', 'min_SNR': 2.0, 'rval_thr': 0.7, 'use_cnn': True, 'min_cnn_thr': 0.8, 'cnn_lowest': 0.1, 'decay_time': 0.4}, 'refit': True}
Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf4442

2023-01-08 01:31:22.359823: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-08 01:31:22.359852: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: hantman-calcium
2023-01-08 01:31:22.359858: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: hantman-calcium
2023-01-08 01:31:22.360004: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2023-01-08 01:31:22.360026: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.161.3
2023-01-08 01:31:22.360033: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.161.3
2023-01-08 01:31:22.360250: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU

performing CNMF
fitting images
refitting
performing eval
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /home/kushalk/caiman_data/model/cnn_model.json
5/5 [==============================] - 0s 11ms/step
Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/2b9dabd8-7d22-4327-8479-aea36bde79cf_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/2b9dabd8-7d22-4327-8479-aea36bde79cf_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/2b9dabd8-7d22-4327-8479-aea36bde79cf_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/2b9dabd8-7d22-4327-8479-aea36bde79cf_cnmf-memmap_d1_170_d2_170_d3_1_order_C

2023-01-08 01:31:25.956403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 01:31:26.039455: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 01:31:26.410378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

...

2023-01-08 01:33:43.710247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 01:33:43.791697: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 01:33:44.162603: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                          Sue_2x_3000_40_-46
input_movie_path    09610b86-917a-41e7-a493-2d036bf44428/09610b86-...
params              {'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...
outputs                                                          None
comments                                                         None
uuid                             f4474d42-f6ae-4fba-a1a5-c181b4b1796e
Name: 9, dtype: object
With params:{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'tsub': 1, 'method_init': 'greedy_roi', 'min_SNR': 2.0, 'rval_thr': 0.7, 'use_cnn': True, 'min_cnn_thr': 0.8, 'cnn_lowest': 0.1, 'decay_time': 0.4}, 'refit': True}


Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
making memmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/f4474d42-f6ae-4fba-a1a5-c181b4b1796e_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap


2023-01-08 01:34:02.991238: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-08 01:34:02.991270: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: hantman-calcium
2023-01-08 01:34:02.991276: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: hantman-calcium
2023-01-08 01:34:02.991406: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2023-01-08 01:34:02.991425: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.161.3
2023-01-08 01:34:02.991430: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.161.3
2023-01-08 01:34:02.991645: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU

performing CNMF
fitting images
refitting
performing eval
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /home/kushalk/caiman_data/model/cnn_model.json
12/12 [==============================] - 0s 10ms/step
Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/f4474d42-f6ae-4fba-a1a5-c181b4b1796e_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/f4474d42-f6ae-4fba-a1a5-c181b4b1796e_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/f4474d42-f6ae-4fba-a1a5-c181b4b1796e_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/f4474d42-f6ae-4fba-a1a5-c181b4b1796e_cnmf-memmap_d1_170_d2_170_d3_1_order

### We now have CNMF outputs

In [135]:
df = df.caiman.reload_from_disk()
df[df["algo"] == "cnmf"]

,algo,item_name,input_movie_path,params,outputs,comments,uuid
1,cnmf,Sue_2x_3000_40_-46,abe66fc1-5303-4b43-952b-ccf8ebe82301/abe66fc1-5303-4b43-952b-ccf8ebe82301-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 't...",{'mean-projection-path': aa43b012-7352-4f6d-906d-43df7d91972e/aa43b012-7352-4f6d-906d-43df7d91972e_mean_projection.n...,None,aa43b012-7352-4f6d-906d-43df7d91972e
2,cnmf,Sue_2x_3000_40_-46,abe66fc1-5303-4b43-952b-ccf8ebe82301/abe66fc1-5303-4b43-952b-ccf8ebe82301-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 'ts...",{'mean-projection-path': 858ae543-70db-48df-975c-9a4d98847234/858ae543-70db-48df-975c-9a4d98847234_mean_projection.n...,None,858ae543-70db-48df-975c-9a4d98847234
3,cnmf,Sue_2x_3000_40_-46,abe66fc1-5303-4b43-952b-ccf8ebe82301/abe66fc1-5303-4b43-952b-ccf8ebe82301-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 't...",{'mean-projection-path': 1f83e8dd-9c9b-4f57-8ee1-6b2e5c9e7afe/1f83e8dd-9c9b-4f57-8ee1-6b2e5c9e7afe_mean_projection.n...,None,1f83e8dd-9c9b-4f57-8ee1-6b2e5c9e7afe
4,cnmf,Sue_2x_3000_40_-46,abe66fc1-5303-4b43-952b-ccf8ebe82301/abe66fc1-5303-4b43-952b-ccf8ebe82301-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 'ts...",{'mean-projection-path': a41bc279-4a98-4f34-8f4e-e19037991fde/a41bc279-4a98-4f34-8f4e-e19037991fde_mean_projection.n...,None,a41bc279-4a98-4f34-8f4e-e19037991fde
5,cnmf,Sue_2x_3000_40_-46,abe66fc1-5303-4b43-952b-ccf8ebe82301/abe66fc1-5303-4b43-952b-ccf8ebe82301-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 't...",{'mean-projection-path': c39a789d-c233-4b4c-89d1-cab1ba6ba251/c39a789d-c233-4b4c-89d1-cab1ba6ba251_mean_projection.n...,None,c39a789d-c233-4b4c-89d1-cab1ba6ba251
6,cnmf,Sue_2x_3000_40_-46,abe66fc1-5303-4b43-952b-ccf8ebe82301/abe66fc1-5303-4b43-952b-ccf8ebe82301-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 'ts...",{'mean-projection-path': 6619cfdc-b4ac-44fc-9fb2-691eb9a73b14/6619cfdc-b4ac-44fc-9fb2-691eb9a73b14_mean_projection.n...,None,6619cfdc-b4ac-44fc-9fb2-691eb9a73b14
7,cnmf,Sue_2x_3000_40_-46,abe66fc1-5303-4b43-952b-ccf8ebe82301/abe66fc1-5303-4b43-952b-ccf8ebe82301-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 't...",{'mean-projection-path': ff63724a-052b-44b7-9421-9daea5f7e380/ff63724a-052b-44b7-9421-9daea5f7e380_mean_projection.n...,None,ff63724a-052b-44b7-9421-9daea5f7e380
8,cnmf,Sue_2x_3000_40_-46,abe66fc1-5303-4b43-952b-ccf8ebe82301/abe66fc1-5303-4b43-952b-ccf8ebe82301-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'ts...",{'mean-projection-path': 96edc395-671c-49b9-8ecf-870a4c4f0c1a/96edc395-671c-49b9-8ecf-870a4c4f0c1a_mean_projection.n...,None,96edc395-671c-49b9-8ecf-870a4c4f0c1a
9,cnmf,Sue_2x_3000_40_-46,abe66fc1-5303-4b43-952b-ccf8ebe82301/abe66fc1-5303-4b43-952b-ccf8ebe82301-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 't...",{'mean-projection-path': 9992fd2c-3a4f-449a-a404-2b1c4bb0772e/9992fd2c-3a4f-449a-a404-2b1c4bb0772e_mean_projection.n...,None,9992fd2c-3a4f-449a-a404-2b1c4bb0772e


In [136]:
# see which batch items completed succcessfully
df[df["algo"] == "cnmf"]["outputs"].apply(lambda x: x["success"])

1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
9    True
Name: outputs, dtype: bool